# Permutations on Ruby

## Classes

### Permutation class

In [115]:
class Permutation
  def initialize(image=(0..4).to_a)
    raise ArgumentError, "Not non-negative integers" unless image.all?{|v| v.is_a?(Integer) && (v >= 0)}
    raise ArgumentError, "Not a permutation of 0 to #{image.size}" unless image.sort == (0..image.size-1).to_a
    @size = image.size
    @image = image
  end
  attr_accessor :size, :image
#-----
  def act(arg)
#    raise ArgumentError, "#{self.show} can not act onto #{arg}" unless image.all?{|v| arg.include?(v)}
    raise ArgumentError, "Not non-negative integers" unless [arg].flatten.all?{|v| v.is_a?(Integer) && (v >= 0)}
    case arg
    when Integer
      return @image[arg].then{|v| (v.nil?) ? arg : v}
    when Array
      return arg.map{|v| self.act(v)}
    else
      raise ArgumentError, "Input an Integer or an Array!"
    end
  end
  def *(aPerm)
    raise ArgumentError, "Not a Permutation" unless aPerm.is_a?(Permutation)
    return Permutation.new(self.act(aPerm.image))
  end
#-----    
  def to_s
    "#{(0..@size-1).to_a}\n#{image}"
  end
  def inspect
    "(" + (0..@size-1).map{|i| "#{i}\u{21a6}#{image[i]}"}.join(',') + ")"
#    "(" + (0..@size-1).map{|i| "\u{21a6}#{image[i]}"}.join(',') + ")"
  end
  def display(before)
    "#{before}  |-#{self.inspect}->  #{self.act(before)}"
  end
end; nil

In [116]:
sigma = Permutation.new([1,2,0,3])
puts sigma.to_s

before = [2,0,3,4,1]
sigma.display(before)

[0, 1, 2, 3]
[1, 2, 0, 3]


"[2, 0, 3, 4, 1]  |-(0↦1,1↦2,2↦0,3↦3)->  [0, 1, 3, 4, 2]"

### Cycle class (Cyclic permutation)

In [182]:
class Cycle < Permutation
  def initialize(arg)
    @seq = arg
    @length = arg.size
    image = (0..arg.max).map do |k|
      ind = arg.index(k)
      (ind.nil?) ? k : arg[(ind+1)%arg.size]
    end
    super(image)
  end
  attr_accessor :length
#-----  
  def to_s
    "(#{@seq.join(',')})"
  end
  def inspect
    "(#{@seq.join("\u{21a6}")}\u{21a6})"
  end
end; nil

In [183]:
c = Cycle.new([2,4,1])

p c.image, c.to_s, c.inspect

c.display([0,2,1,3,4])

[0, 2, 4, 3, 1]
"(2,4,1)"
"(2↦4↦1↦)"


"[0, 2, 1, 3, 4]  |-(2↦4↦1↦)->  [0, 4, 2, 3, 1]"

### Transposition class

In [194]:
class Transposition < Cycle
  def initialize(i,j)
    @pair = [i,j]
    super(@pair)
  end
#-----  
  def inspect
    "(#{@seq.join("\u{21c4}")})"
  end
end; nil

In [195]:
t = Transposition.new(3,4)
puts t.to_s, t.inspect

t.display((0..5).to_a)

(3,4)
(3⇄4)


"[0, 1, 2, 3, 4, 5]  |-(3⇄4)->  [0, 1, 2, 4, 3, 5]"

## Decomposition into Cycles

In [165]:
def decompose(perm)
  seqs = []
  im = perm.image.dup
  while im.size > 0
    seq = [im.first]
    while seq.count(seq.first) == 1
      seq << perm.act(seq.last)
    end
    seq.pop
    seqs << Cycle.new(seq)
    seq.each{|v| im.delete(v)}
  end
  return seqs.sort_by{|a| a.length }
end;nil

In [178]:
v = (0..8).to_a.permutation.to_a.sample
s = Permutation.new(v).tap{|x| p x}

decomp = decompose(s).tap{|x| p x}

arr = (0..6).to_a
decomp.each do |perm|
  p perm.display(arr)
  arr = perm.act(arr)
end; nil

(0↦8,1↦2,2↦3,3↦4,4↦7,5↦6,6↦5,7↦1,8↦0)
[(8↦0↦), (6↦5↦), (2↦3↦4↦7↦1↦)]
"[0, 1, 2, 3, 4, 5, 6]  |-(8↦0↦)->  [8, 1, 2, 3, 4, 5, 6]"
"[8, 1, 2, 3, 4, 5, 6]  |-(6↦5↦)->  [8, 1, 2, 3, 4, 6, 5]"
"[8, 1, 2, 3, 4, 6, 5]  |-(2↦3↦4↦7↦1↦)->  [8, 2, 3, 4, 7, 6, 5]"


## Experiments

In [45]:
s = Permutation.new([1,2,0,3])
t = Transposition.new(3,4)
st = s*t; #puts st.to_s

start = (0..5).to_a

puts st.display(start)

[t,s].map do |perm|
  perm.display(start).tap{|v| start = perm.act(start)}
end.join('mark').sub(/mark(.{15})/, '')

[0, 1, 2, 3, 4, 5]  |-(0↦1,1↦2,2↦0,3↦4,4↦3)->  [1, 2, 0, 4, 3, 5]


"[0, 1, 2, 3, 4, 5]  |-(3⇄4)->  [0, 1, 2, 4, 3, 5] 5]  |-(0↦1,1↦2,2↦0,3↦3)->  [1, 2, 0, 4, 3, 5]"

In [47]:
p t.display(start)

c = Cycle.new([3,4])
p c.display(start)
nil

"[1, 2, 0, 4, 3, 5]  |-(3⇄4)->  [1, 2, 0, 3, 4, 5]"
"[1, 2, 0, 4, 3, 5]  |-(3↦4↦)->  [1, 2, 0, 3, 4, 5]"
